# Contents
- preprocess input review

### Appendix: BiLSTM
- Model
- Train
- Validation
- Test
- Generation

In [1]:
import re
import pandas as pd
import csv
PATH = '/PATH/TO/IMDB_Dataset.csv'
df = pd.read_csv(PATH)
df['sentiment'] = df['sentiment'].map(lambda x: 1 if x== 'positive' else 0)

In [2]:
def preprocess(text):
    
    text = text.lower()
    
    #Replace html
    text = re.sub('(<.*?\>)', ' ', text)
    
    #Convert www.* or https?://* to URL
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',text)

    #Convert @username to AT_USER
    text = re.sub('@[^\s]+','AT_USER',text)

    #Remove additional white spaces
    text = re.sub('[\s]{2,}', ' ', text)

    #Replace #word with word
    text = re.sub(r'#([^\s]+)', r'\1', text)
    
    #trim
    text = text.strip('\'"')
    
    return text

- preprocess

In [3]:
df['review'] = df['review'].map(preprocess)

- tokenize

In [13]:
import spacy
nlp = spacy.load('en_core_web_sm')
nlp.remove_pipe('ner')

def tokenize(paragraph):
    obj_ = ''
    doc = nlp(paragraph)
    tokens = [tok.text for tok in doc]
    for sent in doc.sents:
        obj_ += ' <sos> ' + ' '.join(tok for tok in tokens[sent.start: sent.end] if tok is not None).lower() + ' <eos>'
    return obj_.strip()

df['review'] = df['review'].map(tokenize)

In [12]:
tokenize(df['review'].iloc[2])

'<sos> i thought this was a wonderful way to spend time on a too hot summer weekend , sitting in the air conditioned theater and watching a light - hearted comedy . <eos> <sos> the plot is simplistic , but the dialogue is witty and the characters are likable ( even the well bread suspected serial killer ) . <eos> <sos> while some may be disappointed when they realize this is not match point 2 : risk addiction <eos> <sos> , i thought it was proof that woody allen is still fully in control of the style many of us have grown to love . <eos> <sos> this was the most i \'d laughed at one of woody \'s comedies in years <eos> <sos> ( dare i say a decade ? ) . <eos> <sos> while i \'ve never been impressed with scarlet johanson , in this she managed to tone down her " sexy " image and jumped right into a average , but spirited young woman . <eos> <sos> this may not be the crown jewel of his career , but it was wittier than " devil wears prada " and more interesting than " superman " a great come

- save dataset

In [14]:
fn = '/PATH/TO/SAVE/review_dataset.json'
df.to_json(fn, orient='records', lines=True)

# Load dataset by torchtext

In [1]:
from torchtext import data
import os
from torchtext.data import TabularDataset
import torchtext.vocab as vocab

TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  batch_first=True,)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   preprocessing = lambda x: int(x),
                   is_target=True)

/home/henry/anaconda3/envs/torch37/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [3]:
data = TabularDataset(
    path='./Data/PATH/TO/SAVE/review_dataset.json',
    skip_header=True,
    format='json',
    fields={'review': ('text', TEXT),
            'sentiment': ('label', LABEL)})

In [2]:
## if you have train, test set,
# train_data, validation_data = TabularDataset.splits(
#     path='.', 
#     train='/PATH/TO/SAVE/review_dataset_train.json', 
#     test='/PATH/TO/SAVE/review_dataset_test.json', 
#     format='json',
#     fields={'review': ('text', TEXT),
#             'sentiment': ('label', LABEL)}, skip_header=True)

/home/henry/anaconda3/envs/torch37/lib/python3.7/site-packages/torchtext/data/example.py:13: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


In [4]:
PATH = '/PATH/TO/glove'
FILE = PATH + '/glove.840B.300d.txt'
TEXT.build_vocab(data, vectors=vocab.Vectors(name=FILE,), min_freq=20)

In [7]:
TEXT.vocab.freqs['positive']

874

---
# BiLSTM tutorial part

# Model

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class net(nn.Module):
    def __init__(self, voc_size, emb_size, hidn_size):
        super().__init__()
        self.voc_size = voc_size
        self.emb_size = emb_size
        self.hidn_size = hidn_size
        self.emb = nn.Embedding(num_embeddings=voc_size, embedding_dim=emb_size) # in: (bs, seq_len); out: (bs, seq_len, emb_size)
        self.lstm = nn.LSTM(input_size=emb_size, hidden_size=hidn_size, bidirectional=True) # in: (seq_len, batch, input_size); hidn in: (num_layers * num_directions, batch, hidden_size)
        self.lm_out = nn.Linear(hidn_size*2, voc_size) # input concatenated_out: (seq_len, bs, hidn_dim) -> permute: (bs, seq_len, hidn_dim) 
    
    def init_state(self, bs):
        return (torch.zeros(size=(2, bs, self.hidn_size)), torch.zeros(size=(2, bs, self.hidn_size)))
        
    def forward(self, x, prev_state):
        state_h, state_c = prev_state
        bs = len(x)
        emb = self.emb(x)
        emb = emb.permute(1,0,-1)
        out, (state_h, state_c) = self.lstm(emb, (state_h[:,:bs,:].contiguous(), state_c[:,:bs,:].contiguous()))
        
        forward_out = out[:, :, :self.hidn_size]
        backward_out = out[:, :, :self.hidn_size:]
        concat_h = torch.cat([forward_out[:-2], backward_out[2:]], dim=2) # ignore predict <sos>, <eos>
        final_out = self.lm_out(concat_h.permute(1,0,2)) 
        return final_out.view(final_out.size()[0]*final_out.size()[1], final_out.size()[-1]), (state_h, state_c)

def init_weight(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.xavier_normal_(param.data)
        if 'bias' in name:
            nn.init.constant_(param.data, 0)

def repackage_hidden(h):
    """
    - Wraps hidden states in new Tensors, to detach them from their history.(official doc)
    - Trim backpropagation to avoide vanishing/exploding gradient and release memory by detach tensor from graph
    """
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [6]:
model = net(voc_size=len(TEXT.vocab), emb_size=300, hidn_size=50)
model.apply(init_weight)
model.emb.weight.data.copy_(TEXT.vocab.vectors)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Iterator

### BucketIterator

In [119]:
bs = 5
train_iter, valid_iter, test_iter = data.BucketIterator.splits(datasets=(train_data, validation_data, test_data),
                                                               batch_size=bs,
                                                               device=-1,
                                                               sort_key=lambda x: len(x.text), 
                                                               sort=False,
                                                               repeat=False)

/home/henry/anaconda3/envs/torch37/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


# Train

- initialize gpu

In [209]:
torch.cuda.ipc_collect()
torch.cuda.init()
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |   30418 KB |   91807 KB |   91807 KB |
|       from large pool |       0 B  |   29795 KB |   89386 KB |   89386 KB |
|       from small pool |       0 B  |    1102 KB |    2421 KB |    2421 KB |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |   30418 KB |   91807 KB |   91807 KB |
|       from large pool |       0 B  |   29795 KB |   89386 KB |

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01,  amsgrad=False)
loss_func = nn.CrossEntropyLoss()
model.to(device)
model.train()
epoch=1
for _ in range(epoch):
    total_loss = 0
    
    for i, batch in enumerate(train_iter, 1):
        batch_x, label = map(lambda x: x.to(device), [batch.text, batch.label])
        batch_y = batch_x[:, 1:-1].to(device)
        
        prev_state = model.init_state(bs=bs)
#         prev_state = repackage_hidden(prev_state)
        prev_state = list(map(lambda x: x.to(device), prev_state))
        out, _ = model(batch_x, prev_state)

        optimizer.zero_grad()
        loss = loss_func(out, batch_y.contiguous().view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.detach()

        
        if i%500 == 0:
            print(total_loss/i)
            print('x')
            print(' '.join([TEXT.vocab.itos[id_.item()] for id_ in torch.argmax(out[:100,], dim=1).detach()])) # return first 100 words
            print('y')
            print(' '.join([TEXT.vocab.itos[id_.item()] for id_ in batch_y.contiguous().view(-1).detach()[:100]]))

- save model

In [11]:
f = '0915_biLSTM_1epoch.model'
fn = 'Model/{}.pkl'.format(f)
torch.save(model.state_dict(), fn)

# Validatation

In [401]:
len(test_data),len(valid_iter)

(1499, 350)

- load model

In [124]:
model = net(voc_size=len(TEXT.vocab), emb_size=300, hidn_size=50)
f = '0915_biLSTM_1epoch.model'
fn = 'Model/{}.pkl'.format(f)
model.load_state_dict(torch.load(fn))
model.to(device)

net(
  (emb): Embedding(18604, 300)
  (lstm): LSTM(300, 50, bidirectional=True)
  (lm_out): Linear(in_features=100, out_features=18604, bias=True)
)

In [13]:
def validation_model(model, iterator):
    loss_func = nn.CrossEntropyLoss()
    model.eval()
    with torch.no_grad():
        total_loss = 0
        for i, batch in enumerate(iterator, 1):
            batch_x, label = map(lambda x: x.to(device), [batch.text, batch.label])
            batch_y = batch_x[:, 1:-1].to(device)

            prev_state = model.init_state(bs=len(batch_x))
            prev_state = list(map(lambda x: x.to(device), prev_state))
            out, prev_state = model(batch_x, prev_state)

            loss = loss_func(out, batch_y.contiguous().view(-1))
            total_loss += loss

        print(total_loss/i)
        print('x')
        print(' '.join([TEXT.vocab.itos[id_.item()] for id_ in torch.argmax(out[:100,], dim=1).detach()]))
        print('y')
        print(' '.join([TEXT.vocab.itos[id_.item()] for id_ in batch_y.contiguous().view(-1).detach()[:100]]))

In [14]:
validation_model(model, valid_iter)

tensor(0.1366, device='cuda:0')
x
into this movie , i was a the . . i have always been a bit <unk> and claymation movie . i 've always enjoyed decent animated movies , but marlow was always different to me . but this one caught me by surprise . wallace and gromit are <unk> lovable characters , and it 's a great story with jokes for all ages . there 's the silly <unk> the fart jokes for the kids and the subtle , but over the head of young kids , jokes for us older people . very neat claymation and while the
y
into this movie , i was a bit cautious . i have always been a bit <unk> about claymation movies . i 've always enjoyed decent animated movies , but claymation was always different to me . but this one caught me by surprise . wallace and gromit are extremely lovable characters , and it 's a great story with jokes for all ages . there 's the silly <unk> / fart jokes for the kids and the subtle , but over the head of young kids , jokes for us older people . very neat claymation and wh

# Test

In [15]:
def test_model(model, iterator):
    loss_func = nn.CrossEntropyLoss()
    model.eval()
    with torch.no_grad():
        total_loss = 0
        for i, batch in enumerate(iterator, 1):
            batch_x, label = map(lambda x: x.to(device), [batch.text, batch.label])
            batch_y = batch_x[:, 1:-1].to(device)

            prev_state = model.init_state(bs=len(batch_x))
            prev_state = list(map(lambda x: x.to(device), prev_state))
            out, prev_state = model(batch_x, prev_state)

            loss = loss_func(out, batch_y.contiguous().view(-1))
            total_loss += loss
            
            if 12%i == 0:
                print(total_loss/i)
                print('x')
                print(' '.join([TEXT.vocab.itos[id_.item()] for id_ in torch.argmax(out[:100,], dim=1).detach()]))
                print('y')
                print(' '.join([TEXT.vocab.itos[id_.item()] for id_ in batch_y.contiguous().view(-1).detach()[:100]]))

In [16]:
test_model(model, test_iter)

tensor(0.1717, device='cuda:0')
x
ca n't knock this film too terribly , because it 's obvious midway through the the watching of it that they were trying to make it bad , or ' campy ' if you prefer . anyway , many of the parts they tried to make funny actually are , but often simply for the cheese banter . watch the " space invaders ' game , actually played as real - life ! estevez your head at the bumbling robots ... oh , the <unk> ! and watch the whole thing go way over the top near the end with
y
ca n't knock this film too terribly , because it 's obvious midway through the the watching of it that they were trying to make it bad , or ' campy ' if you prefer . anyway , many of the parts they tried to make funny actually are , but often simply for the cheese factor . watch the ' space invaders ' game , actually played as real - life ! scratch your head at the bumbling robots ... oh , the <unk> ! and watch the whole thing go way over the top near the end with
tensor(0.1478, device='cud

# Generation

- burning

In [221]:
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01,  amsgrad=False)

# loss_func = nn.CrossEntropyLoss()
# model.to(device)

# model.eval()
# with torch.no_grad():
#     prev_state = model.init_state(bs=bs)
#     total_loss = 0
#     for i, batch in enumerate(valid_iter, 1):
#         if i == 50:
#             break
#         batch_x, label = map(lambda x: x.to(device), [batch.text, batch.label])
#         batch_y = batch_x[:, 1:-1].to(device)

#         prev_state = list(map(lambda x: x.to(device), prev_state))
#         out, prev_state = model(batch_x, prev_state)

#         loss = loss_func(out, batch_y.contiguous().view(-1))
#         total_loss += loss

- generation

In [8]:
# 20 samples
len(test_data)
test_gen_data = [test_data.examples[i.item()] for i in torch.randint(0, len(test_data), size=(20,))]

In [9]:
test_gen_dataset = []
for review in test_gen_data:
    temp = [TEXT.vocab.stoi[word] if word in TEXT.vocab.stoi else TEXT.vocab.stoi['<unk>'] for word in review.text]
    test_gen_dataset.append(torch.LongTensor(temp))

In [110]:
def reverse_token(tensor_):
    return ' '.join([TEXT.vocab.itos[i] for i in tensor_])

### Generation
1) `concat[final forward state, final backward state]`

In [199]:
device = 'cpu'
model.eval()
model.to(device)
generated_text = []
# window_size = 100
with torch.no_grad():
    for i, review in enumerate(test_gen_dataset):
        if i == 5:
            break
        nforce_t_text = []
        review = review.to(device)
        half_review = review[:len(review)//2]
        
        prev_state = model.init_state(bs=1)
        prev_state = list(map(lambda x: x.to(device), prev_state))
        
        print('\nfull query\n {}'.format(reverse_token(review)))
        print('\nhalf of query\n {}\n'.format(reverse_token(half_review)))
        
        # input half of sentence
        _, prev_state = model(half_review.view(1,-1), prev_state)
        
        # generated first new token 단어
#         in_ = prev_state[0].permute(0,1,2).view(1,1,-1)
        in_ = prev_state[0].view(1,1,-1)
        out_ = torch.argmax(model.lm_out(in_).squeeze())
        nforce_t_text.append(out_)
        new_seq = torch.cat([half_review, out_.view(1,)])
        
        for _ in range(100):    
            # get final hidden state
            prev_state = model.init_state(bs=1) # init prev state because re-read entire sentence
            prev_state = list(map(lambda x: x.to(device), prev_state))
            _, prev_state = model(new_seq.view(1,-1), prev_state)
            
            # input [concatenated final hidden state]
            in_ = prev_state[0].view(1,1,-1)
            
            # generated new token
            out_ = torch.argmax(model.lm_out(in_).squeeze()) 
            nforce_t_text.append(out_.item())
            
            # concat new words with previous sentence
            new_seq = torch.cat([new_seq, out_.view(1,)])
        
        print('\ngenerated')
        print(' '.join([TEXT.vocab.itos[i] for i in nforce_t_text]))
        generated_text.append(nforce_t_text)
        generated_text.append(nforce_t_text)


full query
 i was pleasantly surprised i quite liked this movie . witty writing ( some " inside " jokes i got , others i did n't - maybe due to actors speaking on top of one another ) , great acting ( notably john <unk> ) , great cameos , interesting and unique directing . i rented it to see jeffrey meek ( very disappointed he was in it such a short time , blink and you 'll miss him ! ) but found the movie remarkably entertaining . i 'll actually watch it again before i send back to netflix . i think actors and wanna - be actors will thoroughly enjoy this movie . the ending is somewhat expected but wish they 'd done something different ( and more positive ) . too bad the movie was n't better received except for in the " festival " market . i suggest it to anyone who loves the acting biz .

half of query
 i was pleasantly surprised i quite liked this movie . witty writing ( some " inside " jokes i got , others i did n't - maybe due to actors speaking on top of one another ) , great act


generated
<unk> honor standards honor to standards platform peace standards honor convention standards honor tragedies standards platform favorites worse platform peace standards honor convention standards honor tragedies standards platform favorites worse platform peace standards honor convention standards honor tragedies standards platform favorites worse platform peace standards honor convention standards honor tragedies standards platform favorites worse platform peace standards honor convention standards honor tragedies standards platform favorites worse platform peace standards honor convention standards honor tragedies standards platform favorites worse platform peace standards honor convention standards honor tragedies standards platform favorites worse platform peace standards honor convention standards honor tragedies standards platform favorites

full query
 i just got home from seeing " radio . " i 've not seen such an inspiring story in a long time . my kids are ages 8 an

---
### Generation
2) final forward state only: `concat[final forward state, zero_vector]`

In [145]:
def filter_unk(tensor_):
    """not support differentiate"""
    tensor_[tensor_ > len(TEXT.vocab)] = 0
    return tensor_

In [196]:
model.eval()
with torch.no_grad():
    for i, batch in enumerate(test_iter, 1):
        if i == 10:
            break
        idx = torch.randint(1, len(b.text), size=(1,)).item()
        batch_x, label = map(lambda x: x.to(device), [batch.text, batch.label])
        batch_x = torch.LongTensor([id_.item() for id_ in batch_x[idx] if id_ != 1])
        batch_x = filter_unk(batch_x)

        half_review = batch_x[:len(batch_x)//2]

        prev_state = model.init_state(bs=1)
        prev_state = list(map(lambda x: x.to(device), prev_state))

        print('\nfull query\n {}'.format(reverse_token(batch_x)))
        print('\nhalf of query\n {}'.format(reverse_token(half_review)))

        # input half of sentence
        _, prev_state = model(half_review.view(1,-1), prev_state)

        # generated first new token 단어
        in_ = torch.cat([prev_state[0][:1,:,:], torch.zeros_like(prev_state[0][1:,:,:])], dim=-1)
        out_ = torch.argmax(model.lm_out(in_).squeeze())
        new_seq = torch.cat([half_review, out_.view(1,)])
        
        nforce_t_text = []
        for _ in range(100):    
            # get final hidden state
            prev_state = model.init_state(bs=1) # init prev state because re-read entire sentence
            prev_state = list(map(lambda x: x.to(device), prev_state))
            _, prev_state = model(new_seq.view(1,-1), prev_state)

            # input [concatenated final hidden state]
            in_ = torch.cat([prev_state[0][:1,:,:], torch.zeros_like(prev_state[0][:1,:,:])], dim=-1)

            # generated new token
            out_ = torch.argmax(model.lm_out(in_).squeeze()) 
            nforce_t_text.append(out_.item())

            # concat new words with previous sentence
            new_seq = torch.cat([new_seq, out_.view(1,)])
            
        print('\ngenerated')
        print(' '.join([TEXT.vocab.itos[i] for i in nforce_t_text]))
        generated_text.append(nforce_t_text)

/home/henry/anaconda3/envs/torch37/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)



full query
 citizen x tells the story of andrei chikatilo , the ripper of <unk> , who killed 52 people in 8 years time , mainly women and children . it shows how the investigation was <unk> by soviet bureaucracy , how hard it was to investigate the crimes . it does the job in such a brilliant way that it will leave no - one untouched . in the beginning it 's perhaps a little bit slow of pace , but it really grabs you as the story unfolds . i can only say that , next to " the silence of the lambs " , this is by far the best movie about a serial killer i 've ever seen . it is very hard to say which actor 's performance stands out above the rest in this movie . stephen rea is really brilliant as the inexperienced forensic expert who is put in charge of the investigation . donald sutherland 's performance as his cynical superior , and the only person in the russian government willing to help him , is as outstanding as rea 's . and what to say about jeffrey <unk> , playing the serial kille


generated
trains discussions grant oneself wet afoul marx labeouf segal eastwood expressions asset regime lights bats expressions mansfield bros trains capshaw mulder borders mayor baddie corporation hamill expressions inaccuracies begins eastwood expressions agatha trains capshaw mulder borders mayor baddie corporation hamill expressions inaccuracies begins eastwood expressions agatha trains capshaw mulder borders mayor baddie corporation hamill expressions inaccuracies begins eastwood expressions agatha trains capshaw mulder borders mayor baddie corporation hamill expressions inaccuracies begins eastwood expressions agatha trains capshaw mulder borders mayor baddie corporation hamill expressions inaccuracies begins eastwood expressions agatha trains capshaw mulder borders mayor baddie corporation hamill expressions inaccuracies begins eastwood

full query
 this is one of the best <unk> of the 60 's put on film . arthur penn , director of bonnie and clyde and little big man , saw tha


generated
expressions discussions kingdom trains reinhold earp deluise midler kingdom trains reinhold earp deluise midler kingdom trains reinhold earp deluise midler kingdom trains reinhold earp deluise midler kingdom trains reinhold earp deluise midler kingdom trains reinhold earp deluise midler kingdom trains reinhold earp deluise midler kingdom trains reinhold earp deluise midler kingdom trains reinhold earp deluise midler kingdom trains reinhold earp deluise midler kingdom trains reinhold earp deluise midler kingdom trains reinhold earp deluise midler kingdom trains reinhold earp deluise midler kingdom trains reinhold earp deluise midler kingdom trains reinhold earp deluise midler kingdom trains reinhold earp deluise midler kingdom trains

full query
 the funniest performance was by <unk> harlow , as matt dillon 's <unk> girlfriend . she was more interesting to me than all the lead actors . this movie got it all wrong ; even the most dependable actress of the century , joan <unk> 

---
### Generation
3) Use model output: Choose last token from `model(cat([half of sentence, <unk><unk>]))`

In [218]:
model.eval()
with torch.no_grad():
    for i, batch in enumerate(test_iter, 1):
        if i == 3:
            break
        idx = torch.randint(1, len(b.text), size=(1,)).item()
        batch_x, label = map(lambda x: x.to(device), [batch.text, batch.label])
        batch_x = torch.LongTensor([id_.item() for id_ in batch_x[idx] if id_ != 1]) # add unk token
        batch_x = filter_unk(batch_x)

        half_review = torch.cat([batch_x[:len(batch_x)//2], torch.LongTensor([0,0])])

        prev_state = model.init_state(bs=1)
        prev_state = list(map(lambda x: x.to(device), prev_state))

        print('\nfull query\n {}'.format(reverse_token(batch_x)))
        print('\nhalf of query\n {}'.format(reverse_token(half_review)))

        # input half of sentence
        out, _ = model(half_review.view(1,-1), prev_state)
        # generated first new token 단어
        out_ = torch.argmax(out[-1,:]).squeeze()
        new_seq = torch.cat([half_review, out_.view(1,)])
        
        nforce_t_text = []
        for _ in range(100):    
            # get final hidden state
            prev_state = model.init_state(bs=1) # init prev state because re-read entire sentence
            prev_state = list(map(lambda x: x.to(device), prev_state))
            out, _ = model(new_seq.view(1,-1), prev_state)
            
            # next token
            out_ = torch.argmax(out[-1,:]).squeeze()

            # generated new token
            nforce_t_text.append(out_.item())

            # concat new words with previous sentence
            new_seq = torch.cat([new_seq, out_.view(1,)])
            
        print('\ngenerated')
        print(' '.join([TEXT.vocab.itos[i] for i in nforce_t_text]))
        generated_text.append(nforce_t_text)


full query
 i love movies . i love independent efforts and major studio productions . i love films with stars and i love those featuring unknowns . i love dramas , comedies , action - adventures , science fiction , mysteries , westerns , any genre except horror . i love foreign films as well as those in english . i love good movies and i even love bad ones , because almost no film ever fails to entertain or amuse on some level . except for " even <unk> get the blues . " when i attended a late - night showing of " <unk> , " i joined an audience of around 10 . less than halfway into it , i alone remained . soon not even i could tolerate the disturbing mess unfolding before my eyes , and i left as well . to this day " <unk> " remains the only movie i have ever walked out of . i do n't quite know how to describe this incoherent , vacuous , trashy , meaningless film , or how to adequately convey its lack of redeeming value . suffice to say that it ranks as one of the worst major films of a


generated
<unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
